In [4]:
%run Utils/Utils.ipynb
%run Utils/Utils_Widgets.ipynb

Servo Motors Ready!
Lidar has started streaming data!
Object recognition model Ready!


In [5]:
# %run Utils/Utils_DepthCam.ipynb
%run Utils/Utils_Camera.ipynb

jetcam library imported
jetcam ready!
torchvision imported
Loading model...
FRCNN model ready!


In [6]:
#!pip install paho.mqtt
%run Utils/Utils_MQTT.ipynb
stepCount = 1

Connected with result #code 0


# Parameters

In [7]:
goal_x_coor = 1.6 #-1.95 #1.6  # in m, measured relative to the initial robot coordinates
goal_y_coor = 5.20 #5.5 #5.20  # in m, measured relative to the initial robot coordinates

running_control_thread = False

getObservation()
print(observation)

[5.441, 1.6, 5.2, 17.103, 0, 0.001, 1.838, 1.835, 1.883, 2.018, 0.001, 2.704, 0.001, 1.076, 0.902, 1.148, 1.18, 1.238, 1.356, 1.561, 1.857, 2.349, 3.493, 7.892, 0.001, 3.35, 3.487, 0.001, 2.064, 1.753, 1.561, 1.373, 4.134, 4.04, 6.601, 5.862, 0.001, 0.001, 0.001, 0.001, 1.114]


In [8]:
# model_path = "Turtlebot3_0331.pth"
model_path = "Turtlebot3_LSTM.pth"
agent = torch.load(model_path)
agent.eval()
print('Model Loaded!')

Model Loaded!


In [9]:
#getObservation()
memories = torch.ones((1,1,256))
masks = torch.ones((1,6))

# Initialize Logging

In [10]:
import logging
import datetime

now = datetime.datetime.now()
dt_string = now.strftime("%Y%m%d__%H_%M_%S")

In [11]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(filename="log_{}.txt".format(dt_string), level=logging.ERROR, format="%(asctime)s %(message)s")
logging.error("{},{},{}".format(real_x, real_y, real_angle))

In [12]:
# Test

# observation = [1]*41

# dists, value_out, memories = agent.get_dist_and_value(vec_inputs=[torch.Tensor(observation).cuda()], vis_inputs=[], masks=masks, memories=memories)
# actions = GetActionDiscrete(agent)  # LSTM
# # actions = GetAction(agent)          # MLP
# actions

# for dist in dists:
#     print(dist.entropy())
    
# for dist in dists:
#     print(dist.probs)
#     dist.probs=dist.probs*5
    
#     print(dist.probs)
    
# for dist in dists:
#     print(dist.probs)
    
# for dist in dists:
#     print(dist.entropy())
    
# actions = []
# for dist in dists:
#     actions.append(dist.sample().cpu().numpy()[0][0])
# print(actions)

# Control

In [13]:
def ControlRobot(debug=False):
    
    global stepCount

    delay_counter = 0
    while True:

        # Get an Observation
        try:
            getObservation()
        except:
            continue

        updateStateWidgets()
        
        # Get Action
        #actions = GetAction(agent)         # Get act ion from MLP
        #actions = GetActionDiscrete(agent)  # Get action from LSTM
        
        actions = GetActionMod(agent)      # Get action from LSTM

        if delay_counter>10:
        
            # If still far away
            if observation[0] > 0.80:
                print(f'Performing action = {actions}')
                act(actions)

            else:
                print("reached goal")
                continue
                
        delay_counter+=1

            

In [14]:
def showDepthcamFeed_TRT_CSI():
    global clockIsVisible, image
    global running_control_thread
    
    while True:

        # Obtain BBoxes + Labels from model
        detections = getDetections(image)
        
        if 33 in detections or 31 in detections or 25 in detections or 85 in detections:
            clockIsVisible =1 
        else:
            clockIsVisible = 0
        
            
        # Run Control Thread
        if running_control_thread == False:
            
            controlThread = Thread(target = ControlRobot)
            controlThread.start()
            running_control_thread = True

In [15]:
display(full_widget)

In [16]:
# Delay Camera thread to make sure it works.
display(image_widget)

In [17]:
# Start camera
execute({'new': camera.value})
camera.observe(execute, names='value')

In [ ]:
# showCSIFeed()
showDepthcamFeed_TRT_CSI()

Performing action = [1, 0]
moved distance =  0.30148454124997176
Performing action = [1, 1]
moved distance =  0.2155441824632945
rotated: -38.88924139064375. Real angle = 38.88924139064375
Performing action = [1, 1]
moved distance =  0.3097289338972241
rotated: -37.06423721526227. Real angle = 75.95347860590601
Performing action = [1, 2]
moved distance =  0.27381425065945725
rotated: 39.15263834599001. Real angle = 36.800840259916
Performing action = [1, 2]
moved distance =  0.2740639814088572
rotated: 39.47887841386722. Real angle = 357.3219618460488
Performing action = [1, 1]
moved distance =  0.2669139010968792
rotated: -39.28785406029425. Real angle = 36.609815906343044
Performing action = [1, 1]
moved distance =  0.2543707698447273
rotated: -39.023194084408246. Real angle = 75.63300999075129
Performing action = [1, 2]
moved distance =  0.26087219669315903
rotated: 39.13340931283065. Real angle = 36.49960067792064
Performing action = [1, 2]
moved distance =  0.2738887145603419
rota